# 집 값 예측하기 실습


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
%matplotlib inline

2023-06-19 17:31:34.744869: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-19 17:31:34.831084: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-19 17:31:34.833035: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-19 17:31:35.983599: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [16]:
data = pd.read_csv("../data/house_price-train.csv")
display(data)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


불필요한 ID 컬럼을 제거해줍니다.

In [17]:
data = data.drop("Id", axis=1)
display(data)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


비 수치형 데이터를 인코딩하여 수치형으로 바꿔줍니다.
OneHotEncoder이라는 것을 사용합니다. 종류가 다양한 경우 종류의 문제에서 수치가 우열의 문제로 바뀔 수 있기 때문입니다.

In [21]:
non_numeric_columns = data.select_dtypes(exclude=np.number).columns
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

for i in non_numeric_columns:
    data[i] = label_encoder.fit_transform(data[i])
display(data)

<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr.csr_matrix'>
<class 'scipy.sparse._csr

수치형 데이터를 정규화 해줍니다.

In [ ]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
numeric_columns = data.select_dtypes(['int64', 'float64']).columns
data[numeric_columns] = min_max_scaler.fit_transform(data[numeric_columns])
display(data)

변수들간의 상관관계를 분석합니다.
독립변수들간에 상관관계가 높으면 중복해서 입력하는 효과가 있기 때문에 데이터를 사용하기 전에 데이터가 정규분포를 따르는지와 더붙어 상관관게 분석을 통해 같이 쓰지 말아야 할 독립 변수들을 파악하고 있는 것이 좋습니다.
아래 코드는 우리가 예측하고자 하는 종속변수인 집값과의 상관계수를 구한 것으로써 이 경우에는 사실 경우에 따라 포함하여야 할지 특정 변수를 뺴야할지 달라질 수 있습니다. 너무 대부분의 독립변수들이 높은 상관관계를 가지게 되면 정답을 뻔하게 알게되고 데이터간 새로운 패턴이나 유의미함을 알 수 없기 떄문입니다.

In [ ]:
np.abs(data[numeric_columns].corr().loc[:,"SalePrice"]).sort_values(ascending=False)

데이터 중에 입력되지 않은 값이나 Nan값이 있는지 확인합니다.

In [ ]:
missing_values = data.isnull().sum().sort_values(ascending=False)
print(missing_values)

직접 Nan이나 빈값, inf값 등등 값들을 포함하는 레코드를 지워도 되지만, 이 예제에서는 빠른 실행을 위해 통째로 특정 컬럼을  제외하고 학습 데이터로 집어 넣겠습니다.

In [ ]:
data_filtered = data.loc[:, ~data.columns.isin(['LotFrontage', 'GarageYrBlt', 'MasVnrArea'])]

In [ ]:
from sklearn.linear_model import LinearRegression

x_train = data_filtered.to_numpy(dtype=np.float32)[:1000, :-1]
y_train = data_filtered.to_numpy(dtype=np.float32)[:1000, -1]

model = LinearRegression()
model.fit(x_train, y_train)

In [7]:
x_test = data_filtered.to_numpy(dtype=np.float32)[1000:, :-1]
y_test = data_filtered.to_numpy(dtype=np.float32)[1000:, -1, np.newaxis]

y_predict = model.predict(x_test)

score = model.score(x_test, y_test)
print(f"score = {score}")

plt.scatter(y_predict, y_test, c='b') # bwr : 파랑-흰색-빨강 색상맵

NameError: name 'data_filtered' is not defined